In [1]:
import pickle
from torch.utils.data import DataLoader, Dataset
from data_utils import tokenizer, SequenceDataset
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn

In [2]:
device=torch.device("mps")
print( torch.backends.mps.is_available())
results_dir = "../data/"
with open(results_dir + 'test.p', 'rb') as handle:
    dataset = pickle.load(handle)

## TODO:Load in the Test Set
test_seq, test_lab = dataset['test_seq'], dataset['test_lab']
test_mask = np.ones(test_seq.shape)
test_dataset = SequenceDataset(test_seq, test_mask, test_lab, device=device)


True


In [3]:

test_loader = DataLoader(test_dataset, shuffle=False, batch_size=4)
# random_indices = random.sample(range(len(original_list)), 10)

for i, data in list(enumerate(test_loader))[-5:]:
    print(data['Class'])

tensor([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]], device='mps:0')
tensor([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]], device='mps:0')
tensor([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]], device='mps:0')
tensor([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]], device='mps:0')
tensor([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]], device='mps:0')


In [4]:
print(len(test_dataset) / 4)

10400.0


In [5]:
def predict_one(sample, model):
    sample = sample[1]
    seq, label = sample['Sequence'], sample['Class']
    logits, attention_scores = model(seq, device, predict=True)
    print(logits, label)
    print(attention_scores)

In [11]:
from model import GenoClassifier
import torch
import torch.nn as nn
tkr = dataset['tokenizer']
# PATH = "/data/Dcode/pranav/genoscanner/data/best_model_params128.pt"
PATH="/Users/pranavmahableshwarkar/CS/NIH/DNATransformerClassifier/data/best_model_params128.pt"
ntokens = len(tkr.kmer2idx.keys())
d_model = 144
d_hidden = 256
nlayers = 6
nhead = 12
dropout = 0.2
num_classes = 3

# Load in the Model and Load the Parameters from Training
loaded_state_dict = torch.load(PATH, map_location=torch.device('cpu'))
model = GenoClassifier(ntokens, d_model, d_hidden, nlayers, nhead, dropout, num_classes) # Create an instance of your model
model_state_dict = model.state_dict()

# Remove the "module." prefix from the keys in the loaded state dictionary
loaded_state_dict = {key.replace("module.", ""): value for key, value in loaded_state_dict.items()}

# Update the model's state dictionary with the loaded state dictionary
model.load_state_dict(loaded_state_dict)
model.to(device)
model.eval()


GenoClassifier(
  (transformer): GeneTransformer(
    (embedding): Embedding(261, 144)
    (pos_encoder): PositionalEncoding(
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (transformer_encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=144, out_features=144, bias=True)
          )
          (linear1): Linear(in_features=144, out_features=256, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
          (linear2): Linear(in_features=256, out_features=144, bias=True)
          (norm1): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.2, inplace=False)
          (dropout2): Dropout(p=0.2, inplace=False)
        )
      )
    )
  )
  (lstm): LSTM(144, 256, num_layers=4, batch_first=True)
  (ff): Lin

In [7]:
list_loader = list(enumerate(test_loader))
# predict_one(list_loader[10000], model)

In [8]:
# train_seq, train_lab = dataset["train_seq"], dataset["train_lab"]
# train_mask = np.ones(train_seq.shape)
# train_dataset = SequenceDataset(train_seq, train_mask, train_lab, device)

counts = {
    0: np.count_nonzero(test_lab == 0),
    1: np.count_nonzero(test_lab == 1),
    2: np.count_nonzero(test_lab == 2)
}

# Print the counts
for value, count in counts.items():
    print(f"Value {value}: {count} occurrences")

Value 0: 5692 occurrences
Value 1: 32548 occurrences
Value 2: 3360 occurrences


In [9]:
# class_weights = torch.FloatTensor([5.46, 1.0, 8.35])
criterion = nn.CrossEntropyLoss()
def predict(loader):
    model.eval()
    total_loss = 0.0
    total_samples = 0
    total_correct = 0

    with torch.no_grad():
        for i, data in enumerate(loader):
            vseqs, vlabels = data["Sequence"], data["Class"]
            voutputs, vattn = model(vseqs, device)  # removed masks
            vloss = criterion(voutputs, vlabels)
            total_loss += vloss
            
            predicted_labels = torch.argmax(voutputs, dim=1)
            true_labels = torch.argmax(vlabels, dim=1)
            total_correct += (predicted_labels == true_labels).sum().item()
            total_samples += vlabels.size(dim=0)

    accuracy = total_correct / total_samples
    average_loss = total_loss / len(loader)
    
#     n_classes = 3  # Number of classes
#     true_labels_bin = label_binarize(true_labels, classes=np.arange(n_classes))

#     # Calculate AUC for each class
#     auc_scores = []
#     for class_index in range(n_classes):
#         # Treat current class as the positive class and the rest as negative
#         class_probs = predicted_probs[:, class_index]
#         class_labels = true_labels_bin[:, class_index]
#         auc = roc_auc_score(class_labels, class_probs)
#         auc_scores.append(auc)

#     # Calculate average AUC across all classes
#     average_?uc = np.mean(auc_scores)
    print(accuracy, average_loss)
    return accuracy, average_loss

In [13]:
print(predict(test_loader))

KeyboardInterrupt: 

In [14]:
from sklearn.model_selection import train_test_split


In [15]:
seq = dataset['test_seq']
lab = dataset['test_lab']
full_dataset = SequenceDataset(seq, np.ones(seq.shape), lab, device=device)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

In [22]:
indices = np.asarray(lab != 1).nonzero()
lab = lab[indices]
seq = seq[indices]
print(lab.shape, seq.shape)

(9052,) (9052, 250)
